# GONZAG cloud example

In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
from gonzag.config import *
from gonzag.io import *
from gonzag.utils import *
from gonzag.mod2sat import *
from gonzag.bilin_mapping import *
from gonzag.spectralysis import *


## Data

In [3]:
import xarray as xr
osn_url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/eNATL60_surface_region_1'
dsmod = xr.open_zarr(osn_url, consolidated=True)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates:
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables:
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosstsst       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [5]:
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
l_griddist = False

In [6]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [7]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T19:39:52Z: Creation
    institution:               CLS, CNES
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Altika
    processing_level:          L3
    product_version:           2019
    project:                   COPERNICUS MARINE ENVIRONMENT MONITORING SERVI...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [8]:
name_ssh_sat='sla_unfiltered'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [9]:
    (date1,date2), (Nts,Ntm) = GetTimeOverlap( dssat , dsmod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time"
   => first and last date:  2013-03-14T05:44:50.445185024 -- 2015-03-31T22:41:36.909480960
 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time_counter"
   => first and last date:  2010-02-01T00:30:00.000000000 -- 2010-04-30T23:30:00.000000000

 *** Earliest/latest dates:
   => for satellite data: 2013-03-14T05:44:50.445185024 2015-03-31T22:41:36.909480960 
   => for model     data: 2010-02-01T00:30:00.000000000 2010-04-30T23:30:00.000000000 

 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-02-01 -- 2014-04-30


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [10]:
  
    ModelGrid = ModGrid( dsmod, date1 , date2 , dsmod, name_lsm_mod, distorded_grid=False )


 *** [GetModelCoor()] Read model latitude (variable is "lat", with 2 dimensions! (763, 611) 

 *** [GetModelCoor()] Read model longitude (variable is "lon", with 2 dimensions! (763, 611) 


 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  0.016657295  degrees 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657295  degrees or  1.8507920147292316  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.99 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2136
       =

In [11]:
SatelliteTrack = SatTrack( dssat, date1, date2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...
 *** [GetSatCoor()] reading "latitude" in dataset ...
   => 4150072 records read...

 *** [GetSatCoor()] reading "longitude" in dataset ...
   => 4150072 records read...


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  10236
       ==> time record indices: 14883510 to 19033581, included

       separated in 87 tracks


In [12]:
MG= ModelGrid
ST= SatelliteTrack

In [13]:
from gonzag.io   import GetModel2DVar, GetSatSSH
from gonzag.bilin_mapping import NearestPoint
from gonzag.bilin_mapping import IDSourceMesh
from gonzag.bilin_mapping import WeightBL
import pandas as pd


(Nj,Ni) = MG.shape

d_found_km = rfactor*MG.HResDeg*deg2km
print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

# Size of the search zoom box:
np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

Nt = ST.size ; # number of satellit observation point to work with here...

if_talk = Nt//nb_talk

Yt=ST.lat.values
Xt=ST.lon.values
Ys=MG.lat.values
Xs=MG.lon.values
src_grid_local_angle=MG.xangle
k_ew_per=MG.EWPer
rd_found_km=d_found_km
np_box_r=np_box_radius
freq_talk=if_talk

sangle = src_grid_local_angle
kewp   = k_ew_per
rfound = rd_found_km
nprad  = np_box_r
(Nj,Ni) = Ys.shape
        
NP = nmp.zeros((1,2)  , dtype=nmp.int64) ; # nearest point
SM = nmp.zeros((1,4,2), dtype=nmp.int64) ; # source mesh
WB = nmp.zeros((1,4))                    ; # weights
vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
vdistance = nmp.zeros(Nt)

[jinit,iinit] = [nprad,nprad] ; # stupid first guess here...


 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940110469238  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...



In [14]:
        one_track=0
        jt1=ST.index_tracks[one_track][0]
        jt2=ST.index_tracks[one_track][1]

        vssh_s = GetSatSSH( ST.file, name_ssh_sat,  kt1=jt1, kt2=jt2 )

 *** [GetSatSSH()] Reading satellite "sla_unfiltered in dataset



In [15]:
print(jt1,jt2)

0 167


In [16]:
vssh_s

<xarray.DataArray 'sla_unfiltered' (time: 168)>
array([ 0.073,  0.057,  0.051,  0.049,  0.047,  0.061,  0.06 ,  0.059,
        0.056,  0.05 ,  0.058,  0.07 ,  0.073,  0.096,  0.105,  0.089,
        0.064,  0.067,  0.084,  0.056,  0.047,  0.074,  0.063,  0.062,
        0.083,  0.05 ,  0.052,  0.071,  0.063,  0.049,  0.049,  0.051,
        0.055,  0.053,  0.041,  0.053,  0.023,  0.046,  0.061,  0.054,
        0.071,  0.067,  0.054,  0.069,  0.078,  0.083,  0.047,  0.049,
        0.066,  0.083,  0.067,  0.056,  0.062,  0.053,  0.06 ,  0.062,
       -0.001,  0.059,  0.031,  0.124,  0.032,  0.017,  0.062,  0.036,
        0.042,  0.052,  0.014,  0.015,  0.038,  0.06 ,  0.053,  0.032,
        0.046,  0.045,  0.034,  0.026,  0.024,  0.034,  0.035,  0.017,
       -0.001, -0.014, -0.013,  0.006,  0.026, -0.012,  0.02 ,  0.025,
        0.003,  0.004,  0.019,  0.019,  0.002,  0.012,  0.014, -0.015,
        0.   , -0.011,  0.005,  0.029,  0.014,  0.002,  0.003,  0.002,
       -0.004,  0.01 ,  0.024,  0.027,  0.023,  0.024,  0.049,  0.023,
        0.028,  0.06 ,  0.073,  0.073,  0.07 ,  0.077,  0.076,  0.061,
        0.098,  0.118,  0.064,  0.062,  0.079,  0.046,  0.038,  0.073,
        0.082,  0.065,  0.093,  0.096,  0.058,  0.087,  0.081,  0.073,
        0.058,  0.062,  0.065,  0.073,  0.042,  0.032,  0.039,  0.061,
        0.058,  0.039,  0.054,  0.072,  0.062,  0.08 ,  0.109,  0.108,
        0.127,  0.168,  0.13 ,  0.122,  0.126,  0.099,  0.104,  0.077,
        0.077,  0.13 ,  0.1  ,  0.058,  0.074,  0.068,  0.081,  0.096])
Coordinates:
    latitude   (time) float64 -69.16 -69.1 -69.05 -69.0 ... -60.18 -60.12 -60.07
    longitude  (time) float64 28.19 28.11 28.04 27.97 ... 19.51 19.48 19.44
  * time       (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2013-0...
Attributes:
    comment:        The sea level anomaly is the sea surface height above mea...
    long_name:      Sea level anomaly not-filtered not-subsampled with dac, o...
    standard_name:  sea_surface_height_above_sea_level
    units:          m